In [10]:
import pandas as pd
import numpy as np
from pathlib import Path
import datetime

import requests
import re
import os
import sys

import pickle
import ffmpeg


from multiprocess import Pool

from tqdm import tqdm

In [2]:
# displaying options
pd.set_option('display.float_format', '{:.20f}'.format)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
# deprecation warning set off
pd.options.mode.chained_assignment = None 

In [3]:
pathlist = ['/nas-slot3/schwurbel1', '/nas-slot4/schwurbel2']

In [4]:
t1 = pd.read_csv('/home/mangermaier/master_thesis_css/transcribed_messages_df.csv.gzip', compression = 'gzip')
t2 = pd.read_csv('/home/mangermaier/master_thesis_css/transcribed_messages_df_1.csv.gzip', compression = 'gzip')
t3 = pd.read_csv('/home/mangermaier/master_thesis_css/transcribed_messages_df_2.csv.gzip', compression = 'gzip')
t4 = pd.read_csv('/home/mangermaier/master_thesis_css/transcribed_messages_df_3.csv.gzip', compression = 'gzip')
t5 = pd.read_csv('/home/mangermaier/master_thesis_css/transcribed_messages_df_4.csv.gzip', compression = 'gzip')
t6 = pd.read_csv('/home/mangermaier/master_thesis_css/transcribed_messages_df_5.csv.gzip', compression = 'gzip')
t7 = pd.read_csv('/home/mangermaier/master_thesis_css/transcribed_messages_df_6.csv.gzip', compression = 'gzip')
t8 = pd.read_csv('/home/mangermaier/master_thesis_css/transcribed_messages_df_7.csv.gzip', compression = 'gzip')

transcribed = pd.concat([t1, t2, t3, t4, t5, t6, t7, t8])

In [5]:
transcribed.shape

(4560, 36)

In [6]:
topic_groups = pd.read_csv('/home/mangermaier/master_thesis_css/selected_groups.csv.gzip', compression = 'gzip')

/tmp/ipykernel_470676/2262508209.py:1: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  topic_groups = pd.read_csv('/home/mangermaier/master_thesis_css/selected_groups.csv.gzip', compression = 'gzip')


In [11]:
test = pd.read_csv("/data/schwurbelarchiv/extracted_information/dataframes/telegram_chats_file_level_w_missing.csv.gzip", compression = 'gzip')

/tmp/ipykernel_1116924/3415011880.py:1: DtypeWarning: Columns (2,6,10,11,12,13,14,15,27) have mixed types. Specify dtype option on import or set low_memory=False.
  test = pd.read_csv("/data/schwurbelarchiv/extracted_information/dataframes/telegram_chats_file_level_w_missing.csv.gzip", compression = 'gzip')


In [2]:
import whisper


In [3]:

#transcription_model_large = whisper.load_model("large")
transcription_model_small = whisper.load_model("small")

In [4]:
#load groupname dictionary
def groupname_dictionary():
    with open('group_name_dictionary.pkl', 'rb') as f:
        res_dict = pickle.load(f)
    return res_dict

In [5]:
def get_filetypes(df):
    
    file_types = df['media_file_type'].combine_first(df['fwd_media_file_type']).unique()
    file_types = file_types[~pd.isnull(file_types)]
    #delete photo, as we do not use it to derive any information yet
    file_types = np.delete(file_types, np.where(file_types == 'photo'))
    
    return file_types

In [6]:
def file_server_dictionary(pathlist):
    server_dict = {}
    for path in pathlist:
        for file in os.listdir(path):
            server_dict.update({file: f"{path}/{file}"})
            
    return server_dict

In [7]:
#deprecated, current version is split in order to parallelize

def file_duration(pathlist, data):
    
    
    ftypes = get_filetypes(data)
    name_dict = groupname_dictionary()
    server_dict = file_server_dictionary(pathlist)
    
    
    for i in tqdm(data['group_name'].unique()):
        #print('check 1')
        for n in ftypes:
            group_df = data[data["group_name"] == i]
            media_df = group_df[group_df["media_file_type"] == n]
            #fwd_media_df = group_df[group_df["fwd_media_file_type"] == n]
            files = list(media_df["media_file"])# + list(fwd_media_df["fwd_media_file"])
            #print('check 2')
            
            for file in tqdm(files):
                try:
                    link = f"{server_dict[name_dict[i]]}/{file}"
                    duration = float(ffmpeg.probe(link)["streams"][0]['duration'])
                    data.loc[data["media_file"] == file, "file_duration"] = duration

                except:
                    continue

    return data

print('deprecated')

deprecated


In [8]:
def get_media_filepaths(pathlist, data):
    
    ftypes = get_filetypes(data)
    name_dict = groupname_dictionary()
    server_dict = file_server_dictionary(pathlist)
    
    link_tuple_list = []
    
    
    for i in data['group_name'].unique():
        #print('check 1')
        for n in ftypes:
            group_df = data[data["group_name"] == i]
            media_df = group_df[group_df["media_file_type"] == n]
            #fwd_media_df = group_df[group_df["fwd_media_file_type"] == n]
            files = list(media_df["media_file"])# + list(fwd_media_df["fwd_media_file"])
            
            for file in files:
                link = f"{server_dict[name_dict[i]]}/{file}"
                link_tuple_list.append((link, i, file))

            
    return link_tuple_list

In [9]:
def get_file_duration(filepath_tuple):
    
    filepath = filepath_tuple[0]
    
    try:
        duration = float(ffmpeg.probe(filepath)["streams"][0]['duration'])
    except:
        duration = float("NaN")
    
    return (duration, filepath_tuple[0], filepath_tuple[1], filepath_tuple[2])   

In [10]:
def pool_file_duration(n_cores, file_tuple_list):
    
    file_metadata = []
    
    file_tuple_list = pd.Series(file_tuple_list).unique().tolist()
    
    pool = Pool(n_cores)
    
    for result in tqdm(
        pool.imap_unordered(func=get_file_duration, iterable=file_tuple_list),
        total=len(file_tuple_list)
        ):
            file_metadata.append(result)


    pool.close()
    
    return file_metadata

In [15]:
file_duration_tuples = pool_file_duration(100, get_media_filepaths(pathlist, topic_groups))

100%|█████████████████████████████████████████████████████████████████████████████| 68092/68092 [33:30<00:00, 33.87it/s]


In [16]:
duration_df = pd.DataFrame(data = file_duration_tuples, columns = ['duration', 'filepath', 'group_name', 'filename'])

In [20]:
duration_df

,duration,filepath,group_name,filename
0,120.62649999999999295142,/nas-slot4/schwurbel2/schwurbelarchiv-NPFJtWIX0R/voice_messages/audio_11825@10-11-2021_00-41-43.ogg,Helfende Q Hände 👐👏,voice_messages/audio_11825@10-11-2021_00-41-43.ogg
1,2.24650000000000016342,/nas-slot4/schwurbel2/schwurbelarchiv-NPFJtWIX0R/voice_messages/audio_11582@03-11-2021_19-06-34.ogg,Helfende Q Hände 👐👏,voice_messages/audio_11582@03-11-2021_19-06-34.ogg
2,528.86649999999997362465,/nas-slot4/schwurbel2/schwurbelarchiv-NPFJtWIX0R/voice_messages/audio_11822@09-11-2021_23-02-35.ogg,Helfende Q Hände 👐👏,voice_messages/audio_11822@09-11-2021_23-02-35.ogg
3,58.34649999999999891998,/nas-slot4/schwurbel2/schwurbelarchiv-NPFJtWIX0R/voice_messages/audio_11823@09-11-2021_23-36-04.ogg,Helfende Q Hände 👐👏,voice_messages/audio_11823@09-11-2021_23-36-04.ogg
4,627.14649999999994633981,/nas-slot4/schwurbel2/schwurbelarchiv-NPFJtWIX0R/voice_messages/audio_11572@03-11-2021_17-50-34.ogg,Helfende Q Hände 👐👏,voice_messages/audio_11572@03-11-2021_17-50-34.ogg
...,...,...,...,...
68087,880.72000000000002728484,/nas-slot3/schwurbel1/schwurbelarchiv-k0gWcZm18e/video_files/video_212@03-06-2021_19-12-51.mp4,5G - ein Verbrechen?! Gibt es chemtrails??,video_files/video_212@03-06-2021_19-12-51.mp4
68088,14.72081599999999923511,/nas-slot3/schwurbel1/schwurbelarchiv-k0gWcZm18e/round_video_messages/file_1@01-06-2021_16-09-25.mp4,5G - ein Verbrechen?! Gibt es chemtrails??,round_video_messages/file_1@01-06-2021_16-09-25.mp4
68089,65.66400000000000147793,/nas-slot3/schwurbel1/schwurbelarchiv-k0gWcZm18e/video_files/video_111@18-11-2020_17-01-45.mp4,5G - ein Verbrechen?! Gibt es chemtrails??,video_files/video_111@18-11-2020_17-01-45.mp4
68090,90.19385400000000174714,/nas-slot3/schwurbel1/schwurbelarchiv-k0gWcZm18e/video_files/video_121@08-12-2020_13-38-15.mp4,5G - ein Verbrechen?! Gibt es chemtrails??,video_files/video_121@08-12-2020_13-38-15.mp4


In [17]:
topic_groups_dur = pd.merge(topic_groups.drop(['Unnamed: 0.1', 'Unnamed: 0', 'transcribed_message'], axis = 1),
                            duration_df,#.drop([], axis = 1),
                            how = 'left',
                            left_on = ['media_file', 'group_name'],
                            right_on = ['filename', 'group_name']
                           )

In [ ]:
link = '/nas-slot3/schwurbel1/schwurbelarchiv-0dYaKpwqbN/video_files/4_5994532154774653081.mp4'
text = transcription_model_small.transcribe(link, fp16=False, language='German')

In [101]:
MIX_group_names = ['BabylonDecoded-Deutsch Chat', 'Weiser mit Kaiser', 'KenFM Ken Jebsen Fanchat Diskussion', 'GEMEINSAMER ANTI CORONALÜGEN - SIEG CHAT! ZUM WIDERSTANDS QUALITÄTSPOSTEN, DISKUSSION UND GEMEINSAMEN GEDANKENAUSTAUSCH', 'BITTEL TV - AUSTAUSCH', 'Alles Ausser Mainstream Chat', 'Klartext reden über Deutschland']

In [12]:
def transcription(dataframe_containing_filepath):
    
    data = dataframe_containing_filepath
    
    for link in tqdm(data['filepath']):
        try:
            text = transcription_model_small.transcribe(link, fp16=False, language='German')
            transcribed = text.get("text")
            data.loc[data['filepath'] == link, 'transcribed_message'] = transcribed
        except:
            continue
        
    return data


dur_sample = topic_groups_trans.query(
    'duration > 60 and duration < 500 and transcribed_message.isna()').groupby(
    ["group_name"]).sample(
    frac = 0.2,random_state = 42)

#check the overall length of files per group in minutes
#dur_sample.groupby(["group_name"])["duration"].sum()/60



In [ ]:
start = datetime.datetime.now()
transcribed = transcription(dur_sample)
end = datetime.datetime.now()
print(end-start)

#increase nomenklatur
transcribed.to_csv('transcribed_messages_df_12.csv.gzip', compression = 'gzip')

  0%|                                                                                           | 0/481 [00:00<?, ?it/s]

In [11]:
topic_groups_trans['transcribed_message'] = topic_groups_trans['transcribed_message'].combine_first(transcribed['transcribed_message'])

In [13]:
dur_sample.shape

(481, 37)

In [58]:
topic_groups_trans.to_csv('/home/mangermaier/master_thesis_css/selected_groups_with_transcriptions.csv.gzip', compression = 'gzip')

In [4]:
topic_groups_trans = pd.read_csv('/home/mangermaier/master_thesis_css/selected_groups_with_transcriptions.csv.gzip', compression = 'gzip')

/tmp/ipykernel_802818/1692778980.py:1: DtypeWarning: Columns (11,12,28,33,34,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  topic_groups_trans = pd.read_csv('/home/mangermaier/master_thesis_css/selected_groups_with_transcriptions.csv.gzip', compression = 'gzip')


In [32]:
topic_groups_trans.query(
    'message.notna()'
).shape

(2735348, 35)

In [86]:
transcribed = transcribed.set_index('Unnamed: 0')

In [34]:
topic_groups_trans.query('transcribed_message.notna()').shape

(5115, 35)

In [35]:
topic_groups_trans.query('duration > 60 and duration < 180 and transcribed_message.isna()').shape


(1807, 35)

In [174]:
transcribed.shape

(506, 32)

In [20]:
topic_groups_trans.columns

Index(['uid', 'mid_message', 'posting_date', 'message', 'fwd_message',
       'fwd_posting_date_message', 'mid_file', 'posting_date_file', 'link_url',
       'media_file', 'media_file_type', 'fwd_link_url', 'fwd_media_file',
       'fwd_media_file_type', 'fwd_posting_date_file', 'group_name',
       'posting_date_author', 'author', 'fwd_author', 'day', 'week', 'weekday',
       'message_hash', 'fwd_message_hash', 'website', 'message_id',
       'replied_to', 'year', 'month', 'day_of_year', 'duration', 'filepath',
       'filename', 'filename_if_joined', 'transcribed_message'],
      dtype='object')

In [24]:
transcribed.columns

Index(['Unnamed: 0', 'duration', 'group_name', 'filename', 'filepath',
       'transcribed_message', 'uid', 'mid_message', 'posting_date', 'message',
       'fwd_message', 'fwd_posting_date_message', 'mid_file',
       'posting_date_file', 'link_url', 'media_file', 'media_file_type',
       'fwd_link_url', 'fwd_media_file', 'fwd_media_file_type',
       'fwd_posting_date_file', 'posting_date_author', 'author', 'fwd_author',
       'day', 'week', 'weekday', 'message_hash', 'fwd_message_hash', 'website',
       'message_id', 'replied_to', 'filename_if_joined'],
      dtype='object')

In [18]:
#probably deprecated cause of combine_first function
trans = transcribed[list(t1.columns)]
topic_groups_trans = pd.merge(topic_groups_dur.drop([], axis = 1),
                            trans.drop(['Unnamed: 0', 'duration', 'filepath'], axis = 1),
                            how = 'left',
                            left_on = ['media_file', 'group_name'],
                            right_on = ['filename', 'group_name'], 
                            suffixes = ('', '_if_joined')
                           )

In [135]:
transcribed.columns

Index(['Unnamed: 0', 'duration', 'group_name', 'filename', 'filepath',
       'transcribed_message'],
      dtype='object')

In [71]:
topic_groups_enh[topic_groups_enh['transcribed_message'] != topic_groups_enh['transcribed_message']]

,uid,mid_message,posting_date,message,fwd_message,fwd_posting_date_message,mid_file,posting_date_file,link_url,media_file,...,weekday,message_hash,fwd_message_hash,website,message_id,replied_to,duration,filename,filepath,transcribed_message
0,209982-6408240263544091615,209982.0,2021-11-09 22:50:15+00:00,Oki 😃,NaN,NaN,209982.0,2021-11-09 22:50:15+00:00,messages199.html#go_to_message209980,NaN,...,2.0,-3.596942e+18,NaN,messages199.html,209982.0,209980-6408240263544091615,NaN,NaN,NaN,NaN
1,209983-6408240263544091615,209983.0,2021-11-09 22:50:18+00:00,Von weiblich bis männlich,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,-7.438051e+18,NaN,NaN,209983.0,NaN,NaN,NaN,NaN,NaN
2,209984-6408240263544091615,209984.0,2021-11-09 22:50:23+00:00,Von direkt zu zurückhaltend,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,-5.141813e+18,NaN,NaN,209984.0,NaN,NaN,NaN,NaN,NaN
3,209985-6408240263544091615,209985.0,2021-11-09 22:50:28+00:00,Wie bei WOW,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,4.179569e+18,NaN,NaN,209985.0,NaN,NaN,NaN,NaN,NaN
4,209986-6408240263544091615,209986.0,2021-11-09 22:50:47+00:00,Neuer Charakter und XP sammeln,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,3.772441e+18,NaN,NaN,209986.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3258423,23042104945287925667135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3258424,23180104945287925667135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3258425,23198104945287925667135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3258426,23213104945287925667135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
